In [1]:
from sentence_transformers import SentenceTransformer, models, util
from nltk import word_tokenize
from collections import defaultdict, Counter
from itertools import chain, combinations
import matplotlib.pyplot as plt
import sbert_training
import pandas as pd
import copy
import torch
import sys
import pickle
import random
from tqdm.notebook import tqdm_notebook

In [2]:
def save_with_pickle(path, data):
    with open(path, "wb") as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_from_pickle(path):
    data = None
    with open(path, "rb") as handle:
        data = pickle.load(handle)
    return data

In [3]:
word_tokenizer = word_tokenize
device = "cuda:0" if torch.cuda.is_available() else "cpu"
repo_dir = "/home/marcelbraasch/PycharmProjects/argmining-21-keypoint-analysis-sharedtask-code-2/"
model_path = "/home/marcelbraasch/PycharmProjects/argmining-21-keypoint-analysis-sharedtask-code/model"
model = SentenceTransformer(model_path)
closed_class_words_path = "./Data/closed.pkl"
closed_class_words = load_from_pickle(closed_class_words_path)

# Load data, unique topics and arguments
mappings_path = "./Data/arg_to_dropped_mapping.pkl"
data_path = "./Data/gold_labels_and_prediction_scores.pkl"
word_importance_path = "./Data/word_importance.pkl"
leave_one_out_path = "./Data/leave_one_out.pkl"

data = load_from_pickle(data_path)
word_importances = load_from_pickle(word_importance_path)
predictions = data["dev"]["predictions"]
arguments = predictions["argument"].unique()
topics = predictions["topic"].unique()
key_points = predictions["key_point"].unique()

2022-02-07 22:59:58 - Load pretrained SentenceTransformer: /home/marcelbraasch/PycharmProjects/argmining-21-keypoint-analysis-sharedtask-code/model
2022-02-07 22:59:59 - Use pytorch device: cuda


In [4]:
def compute_score(arg, kp, model):
    arg = model.encode(arg, show_progress_bar=False),
    kp = model.encode(kp, show_progress_bar=False)
    return float(util.pytorch_cos_sim(arg, kp))

In [5]:
def create_gold_labels_and_prediction_scores(model, path):

    data = defaultdict(dict)
    for subset in ["dev", "train"]:

        # Load files
        arguments_file = repo_dir + f"data/kpm_data/arguments_{subset}.csv"
        key_points_file = repo_dir + f"data/kpm_data/key_points_{subset}.csv"
        labels_file = repo_dir + f"data/kpm_data/labels_{subset}.csv"
        arguments_df = pd.read_csv(arguments_file)
        key_points_df = pd.read_csv(key_points_file)
        labels_df = pd.read_csv(labels_file)

        # Get gold standard
        positive_labels_df = labels_df.loc[labels_df["label"] == 1]
        gold_standard = pd.merge(positive_labels_df, key_points_df, how="inner", on="key_point_id")
        gold_standard = pd.merge(gold_standard, arguments_df, how="inner", on=["arg_id","topic", "stance"])
        gold_standard = gold_standard.rename(columns={"label": "score"})
        data[subset]["gold_standard"] = gold_standard
        
        # Within a topic map every key-point to every argument
        arg_to_kps = {topic: pd.merge(arguments_df.loc[arguments_df["topic"] == topic][["argument"]].drop_duplicates(),
                              key_points_df.loc[key_points_df["topic"] == topic][["key_point"]].drop_duplicates(),
                              how="cross") for topic in arguments_df["topic"].unique()}

        # Create predictions
        mappings = []
        for topic, arg_kps_mapping in arg_to_kps.items():
            arg_kps_mapping['score'] = arg_kps_mapping.apply(lambda row: compute_score(row["argument"], row["key_point"], model), axis=1)
            arg_kps_mapping['topic'] = topic
            arg_kps_mapping = arg_kps_mapping[["topic", "argument", "key_point", "score"]]
            mappings.append(arg_kps_mapping)
        predictions = pd.concat(mappings, axis=0)
        data[subset]["predictions"] = predictions

    save_with_pickle(path, data)

In [6]:
"""
1. Analyze per kp their respective argument and check how similar the arguments are
2. Analyze for how many arg-kp pairs the most prevalent words are occuring in both
3. Analyze bad predictions to maybe understand why theyre wrong, or what made the model do the presumably right, but incorrect, prediction
"""

'\n1. Analyze per kp their respective argument and check how similar the arguments are\n2. Analyze for how many arg-kp pairs the most prevalent words are occuring in both\n3. Analyze bad predictions to maybe understand why theyre wrong, or what made the model do the presumably right, but incorrect, prediction\n'

In [7]:
def powerset(iterable):
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))

def _create_dropped_combinations(argument, drop_size=4):
    tokens = word_tokenizer(argument)
    samples = []
    lexical_mask = [1 if x not in closed_class_words else 0 for x in tokens]
    lexical_indices = [i for i, x in enumerate(lexical_mask) if x]
    lexical_indices_combinations = powerset(lexical_indices)
    lexical_indices_combinations = [x for x in lexical_indices_combinations
                                    if len(x)<=drop_size][1:]
    for combination in lexical_indices_combinations:
        combination = list(combination)
        combination.sort(reverse=True)
        new_arg = copy.deepcopy(tokens)
        dropped_words = [new_arg.pop(index) for index in combination]
        sample = {"dropped": dropped_words,
                  "new_arg": " ".join(new_arg),
                  "amount_dropped": len(combination),
                  "indices": combination}
        samples.append(sample)
    return samples

def create_dropped_combinations(arguments):
    return {argument:_create_dropped_combinations(argument)
            for argument in tqdm_notebook(arguments)}

In [8]:
class Importance:

    def __init__(self):
        self.counter = 0
        self.scores = []

    def update(self, score):
        self.counter += 1
        self.scores.append(score)

    def get(self):
        return sum(self.scores) / self.counter

In [9]:
def compute_word_importance(argument, key_point, arg_to_dropped, model):
    reference = compute_score(argument, key_point, model)
    dropped = arg_to_dropped[argument][:750]
    word_to_importance = defaultdict(Importance)
    for example in tqdm_notebook(dropped):
        dropped_words, new_argument, amount_dropped, indices = example.values()
        new_score = compute_score(new_argument, key_point, model)
        difference = reference - new_score
        for word in dropped_words:
            word_to_importance[word].update(difference)
    return {word: importance.get() for word, importance in word_to_importance.items()}

In [10]:
def compute_word_importances_of_all_arg_kps():
        args_kps = predictions
        arg_to_dropped = create_dropped_combinations(args_kps["argument"].unique())
        args_kps["important_words"] = args_kps.apply(lambda row: compute_word_importance(row["argument"], row["key_point"], arg_to_dropped, model), axis=1)
        save_with_pickle("word_importance.pkl", args_kps)

In [11]:
def create_leave_one_out_for_args():
    loo = []
    for topic in topics:
        # Get all unique key points for a specific topic
        key_points = predictions.loc[predictions['topic'] == topic]["key_point"].unique()
        for argument in tqdm_notebook(arguments):

            # Get the top n key points corresponding to current argument
            top_n = word_importances.loc[word_importances["argument"]==argument] \
                                    .sort_values(by=["score"], ascending=False).head(5)
            df = pd.DataFrame()
            for i, row in enumerate(top_n.iterrows()):
                topic, argument, key_point, score, importances = row[1]
                # Extract word importance scores
                importances = pd.DataFrame.from_dict({x:[y] for x,y in importances.items()}) \
                                .transpose().reset_index() \
                                .rename(columns={"index":f"words_{i}", 0:f"importance_{i}"})
                importances.insert(0, "score", score)
                importances.insert(0, "key_point", key_point)
                df = pd.concat((df, importances), axis=1)
            df.insert(0, 'argument', argument)
            loo.append(df)
    save_with_pickle("./Data/leave_one_out.pkl", loo)

In [12]:
def create_kp_to_its_args():
    argmax_kps = word_importances[word_importances.groupby(['topic',"argument"])['score'].transform(max) == word_importances['score']]
    saved_word_rankings = {}
    for key_point in key_points:
        current_kp = argmax_kps.loc[argmax_kps["key_point"]==key_point]
        counter = defaultdict(int)
        for mapping in current_kp["important_words"]:
            top_5 = {k:v for i, (k,v) in
                     enumerate(sorted(mapping.items(), key=lambda x: x[1], reverse=True))
                     if i <= 5}
            for word in top_5.keys():
                counter[word] += 1
        counter = {k:v for i,(k,v) in enumerate(sorted(counter.items(), key=lambda x: x[1], reverse=True)) if i <= 10}
        saved_word_rankings[key_point] = counter
    return saved_word_rankings

In [13]:
# Part 1: Leave one out
loo = load_from_pickle(leave_one_out_path)

if False:
    for i, sheet in enumerate(loo):
        sheet.to_excel(f"./Results/LOO/sheet_{i}.xlsx")

In [14]:
# Part 2: key-point argument mappoing
if False:
    kptia = create_kp_to_its_args()
    for kp, important_words in kptia.items():
        s = f"{kp}\n"
        for word, occurence in important_words.items():
            s += f"{word}\t{occurence}\n"
        s += "\n"
        with open("kps_importances.txt", "a") as f:
            f.write(s)

In [15]:
from openpyxl import load_workbook

def get_stuff(l):
    new_df = pd.DataFrame()
    argument = l.iloc[:,0][0]
    scores = pd.concat([(l.iloc[:,(4*i)]) for i in range(1,6)], axis=1)
    words = l.iloc[:,3]
    df = pd.concat((words,scores),axis=1)
    kps = [x[1] for x in [l.iloc[:,(4*i-3)] for i in range(1,6)]]
    df = df.rename(columns={f"importance_{i}":kps[i] for i in range(len(kps))})
    df = df.rename(columns={"words_0":"words"})
    topic = list(predictions.loc[predictions["key_point"]==kps[0]]["topic"])[0]
    reference_scores = [x[1] for x in [l.iloc[:,(4*i-2)] for i in range(1,6)]]
    return topic, argument, df, reference_scores

# for i in tqdm_notebook(range(len(loo))):
#     topic, argument, df, reference_scores = get_stuff(loo[i])
#     name_excel = f"./Results/LOO/sheet_{i}.xlsx"
#     name_metad = f"./Results/LOO/sheet_{i}.txt"
#     if i in [0, 1283, 1690, 3306]:
#         print(reference_scores)
    # df.to_excel(name_excel)
    # with open(name_metad, "w") as f:
    #     f.write(f"Topic: {topic}\nArgument: {argument}")

In [16]:
s = 0

2022-02-07 23:35:13 - Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/home/marcelbraasch/PycharmProjects/argmining-21-keypoint-analysis-sharedtask-code/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_38339/938663359.py", line 1, in <module>
    s = 0
  File "/tmp/ipykernel_38339/938663359.py", line 1, in <module>
    s = 0
  File "/home/marcelbraasch/.local/share/JetBrains/Toolbox/apps/PyCharm-P/ch-0/213.5744.248/plugins/python/helpers/pydev/_pydevd_bundle/pydevd_frame.py", line 747, in trace_dispatch
    self.do_wait_suspend(thread, frame, event, arg)
  File "/home/marcelbraasch/.local/share/JetBrains/Toolbox/apps/PyCharm-P/ch-0/213.5744.248/plugins/python/helpers/pydev/_pydevd_bundle/pydevd_frame.py", line 144, in do_wait_suspend
    self._args[0].do_wait_suspend(*args, **kwar

TypeError: object of type 'NoneType' has no len()